# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


In [2]:
ws = Workspace.from_config()

experiment_name = 'capstone-prj-automl'

experiment=Experiment(ws, experiment_name)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster_name = 'capstone-prj-cc'

try:
    compute_target = ComputeTarget(workspace=ws, name = compute_cluster_name)
    print('Found the compute cluster')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found the compute cluster


In [4]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart-failure-dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("dataset found!")

else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

Uploading an estimated of 1 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy", "experiment_timeout_minutes":30, "enable_early_stopping":True, "n_cross_validations":3,"max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name = 'DEATH_EVENT',**automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-prj-automl,AutoML_3a540293-1fab-41d6-bb16-0f420de59867,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

remote_run.wait_for_completion()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_3a540293-1fab-41d6-bb16-0f420de59867',
 'target': 'capstone-prj-cc',
 'status': 'Completed',
 'startTimeUtc': '2021-04-15T15:17:36.498353Z',
 'endTimeUtc': '2021-04-15T15:42:46.537826Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'capstone-prj-cc',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-prj-automl","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-142780","workspace_name":"quick-starts-ws-142780","region":"southcentralus","compute_target":"capstone-prj-cc","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"va

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, model = remote_run.get_output()
print(best_run)
print(model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: capstone-prj-automl,
Id: AutoML_3a540293-1fab-41d6-bb16-0f420de59867_52,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               min_child_weight=1,
                                                                                               missing=nan,
                                    

In [14]:
best_run_metrics


{'AUC_weighted': 0.89877350641388,
 'AUC_micro': 0.9156128236344453,
 'average_precision_score_macro': 0.8978807650590387,
 'average_precision_score_micro': 0.9140015953955486,
 'accuracy': 0.8796396396396396,
 'recall_score_micro': 0.8796396396396396,
 'weighted_accuracy': 0.9108661159004884,
 'average_precision_score_weighted': 0.9103767775506295,
 'precision_score_macro': 0.8911759617899969,
 'AUC_macro': 0.8987735064138799,
 'recall_score_macro': 0.8414386058510314,
 'norm_macro_recall': 0.6828772117020626,
 'balanced_accuracy': 0.8414386058510314,
 'matthews_correlation': 0.730901926537672,
 'log_loss': 0.3877974148440746,
 'f1_score_weighted': 0.8754517786844236,
 'precision_score_weighted': 0.8838205329199479,
 'precision_score_micro': 0.8796396396396396,
 'f1_score_macro': 0.8585837537795168,
 'recall_score_weighted': 0.8796396396396396,
 'f1_score_micro': 0.8796396396396395,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_3a540293-1fab-41d6-bb16-0f420de59867_52/

In [16]:
# Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='AutoML_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['accuracy'],
                                            })

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service